# Example II.1.5

MULTI FACTOR MODEL
Decomposition of portfolio risk into systematic risk and specific risk for a portfolio of stocks exposed to multiple risk factors


In [43]:
#Import statements
import os
import sys
import pandas as pd
import numpy as np
import math

sys.path.append(os.path.abspath('/Users/yashwantbhambhani/Documents/SHWAYB/FSTK'))

from fstk_utilities_pkg.portfolio_manager.portfolio import *
from fstk_utilities_pkg.instruments.equity import *
from fstk_utilities_pkg.data_access_utilities.data_access_util import *
from fstk_utilities_pkg.risk_manager.equity_risk import *
from fstk_utilities_pkg.common_utilities.array_util import *

Specify the folder structure where file is located

In [44]:
volume = "vol2"
chapter = "chap01"
example = "eg05"

Set Name of the hypothetical portfolio

In [45]:
portfolio_name = "port_" + volume + "_" + chapter + "_" + example

Declare variables

In [46]:
obj_portfolio = portfolio(portfolio_name)

obj_df_risk_factors_data = DataAsDataFrame(DatasetTypeList.DATAFRAME)
obj_df_equity_instruments_data = DataAsDataFrame(DatasetTypeList.DATAFRAME)
obj_df_stocks_betas_data = DataAsDataFrame(DatasetTypeList.DATAFRAME)
obj_df_portfolio_data = DataAsDataFrame(DatasetTypeList.DATAFRAME)

#### Load data from the input file
##### Portfolio is comprised of 3 stocks that are sensitive to two risk factors (Index1 and Index2)

In [47]:
df_risk_factors_data = obj_df_risk_factors_data.getData(s_volume=volume, s_chapter=chapter, s_example=example, \
                        s_source_filename="Risk_Factors", s_source_type="csv", verbose=False)

df_equity_instruments_data = obj_df_equity_instruments_data.getData(s_volume=volume, s_chapter=chapter,\
                        s_example=example, s_source_filename="Equity_Instruments", s_source_type="csv", verbose=False)

df_stocks_beta_data = obj_df_stocks_betas_data.getData(s_volume=volume, s_chapter=chapter, s_example=example, \
                        s_source_filename="Stocks_Betas", s_source_type="csv", verbose=False)                                

df_portfolio_data = obj_df_portfolio_data.getData(s_volume=volume, s_chapter=chapter, s_example=example, \
                        s_source_filename="Portfolio", s_source_type="csv", verbose=False)

INFO:fstk_utilities_pkg.data_access_utilities.data_access_util:vol2chap01eg05Risk_Factorscarol_alexander_bookcsv
INFO:fstk_utilities_pkg.data_access_utilities.data_access_util:vol2chap01eg05Equity_Instrumentscarol_alexander_bookcsv
INFO:fstk_utilities_pkg.data_access_utilities.data_access_util:vol2chap01eg05Stocks_Betascarol_alexander_bookcsv
INFO:fstk_utilities_pkg.data_access_utilities.data_access_util:vol2chap01eg05Portfoliocarol_alexander_bookcsv


#### Setting universe of risk factors

In [48]:
arr_risk_factors = []

for risk_factor in df_risk_factors_data.index:
    obj_risk_factor = equity_risk_factor(risk_factor,\
                        df_risk_factors_data.ix[risk_factor]['Asset Class'])    
    arr_risk_factors.append(obj_risk_factor)

#### Setting universe of equity instruments

In [49]:
arr_equity_instruments = []

for equity_instrument in df_equity_instruments_data.index:
    obj_equity = equity_valuation_multi(equity_instrument)
    for risk_factor in arr_risk_factors:
        if risk_factor.getAssetClass() == risk_factor_asset_class.EQT:
            obj_equity.addRiskFactor(risk_factor)
            if (risk_factor.getRiskFactorName() == 'Index1'):
                obj_equity.setRiskFactorSensValue(risk_factor.getRiskFactorName(),\
                        df_stocks_beta_data.ix[equity_instrument]['Beta 1'])
            elif (risk_factor.getRiskFactorName() == 'Index2'):
                obj_equity.setRiskFactorSensValue(risk_factor.getRiskFactorName(),\
                        df_stocks_beta_data.ix[equity_instrument]['Beta 2'])
    arr_equity_instruments.append(obj_equity)

INFO:fstk_utilities_pkg.instruments.equity:Risk Factor To Be Added: Index1
INFO:fstk_utilities_pkg.instruments.equity:Risk Factor Sensitivity to be setIndex10.2
INFO:fstk_utilities_pkg.instruments.equity:Risk Factor To Be Added: Index2
INFO:fstk_utilities_pkg.instruments.equity:Risk Factor Sensitivity to be setIndex21.2
INFO:fstk_utilities_pkg.instruments.equity:Risk Factor To Be Added: Index1
INFO:fstk_utilities_pkg.instruments.equity:Risk Factor Sensitivity to be setIndex10.9
INFO:fstk_utilities_pkg.instruments.equity:Risk Factor To Be Added: Index2
INFO:fstk_utilities_pkg.instruments.equity:Risk Factor Sensitivity to be setIndex20.2
INFO:fstk_utilities_pkg.instruments.equity:Risk Factor To Be Added: Index1
INFO:fstk_utilities_pkg.instruments.equity:Risk Factor Sensitivity to be setIndex11.3
INFO:fstk_utilities_pkg.instruments.equity:Risk Factor To Be Added: Index2
INFO:fstk_utilities_pkg.instruments.equity:Risk Factor Sensitivity to be setIndex20.7


#### Creating a new portfolio

In [50]:
obj_portfolio = portfolio(portfolio_name)

#### Adding positions to the portfolio

In [51]:
for portfolio_item in df_portfolio_data.index:
    #Get the corresponding equity object from the array of equity objects
    for equity_obj in arr_equity_instruments:
        if equity_obj.getIdentifier() == portfolio_item:
            obj_position = equity_position(equity_obj, position_type.EQT,\
                                df_portfolio_data.ix[portfolio_item]['Weight'],
                                df_portfolio_data.ix[portfolio_item]['Weight'])
            obj_portfolio.addPosition(obj_position)

arr_portfolio_positions = obj_portfolio.getPositions()
arr_portfolio_positions_betas = obj_portfolio.getPositionsBetas()

INFO:fstk_utilities_pkg.portfolio_manager.portfolio:<type 'instance'>
INFO:fstk_utilities_pkg.instruments.equity:Retrieving sens value for Stock1
INFO:fstk_utilities_pkg.instruments.equity:Retrieving sens value for Stock1
INFO:fstk_utilities_pkg.portfolio_manager.portfolio:<type 'instance'>
INFO:fstk_utilities_pkg.instruments.equity:Retrieving sens value for Stock2
INFO:fstk_utilities_pkg.instruments.equity:Retrieving sens value for Stock2
INFO:fstk_utilities_pkg.portfolio_manager.portfolio:<type 'instance'>
INFO:fstk_utilities_pkg.instruments.equity:Retrieving sens value for Stock3
INFO:fstk_utilities_pkg.instruments.equity:Retrieving sens value for Stock3


       risk_factor  risk_factor_sens
Index1      Index1               0.2
Index2      Index2               1.2
       risk_factor  risk_factor_sens
Index1      Index1               0.2
Index2      Index2               1.2
       risk_factor  risk_factor_sens
Index1      Index1               0.9
Index2      Index2               0.2
       risk_factor  risk_factor_sens
Index1      Index1               0.9
Index2      Index2               0.2
       risk_factor  risk_factor_sens
Index1      Index1               1.3
Index2      Index2               0.7
       risk_factor  risk_factor_sens
Index1      Index1               1.3
Index2      Index2               0.7


#### Filtering correlations data to get correlation matrix for stocks in the portfolio
##### This information should be cached

In [52]:
obj_db = Db()
df_all_risk_factors_correlations = \
                        obj_db.getAllRiskFactorsCorrelations()

INFO:fstk_utilities_pkg.data_access_utilities.data_access_util:vol2chap01eg05Risk_Factors_Correlationcarol_alexander_bookcsv


#### Retrieve all the correlations for the risk factors of the portfolio

In [53]:
lst_portfolio_all_risk_factors = obj_portfolio.getAllRiskFactorsNames()
row_filter = lst_portfolio_all_risk_factors
column_filter = lst_portfolio_all_risk_factors                        
df_portfolio_stocks_risk_factors_correlations = \
            df_all_risk_factors_correlations.ix[row_filter][column_filter]
lst_portfolio_stocks_risk_factors_correlation = \
            df_portfolio_stocks_risk_factors_correlations.values.tolist()

#### Calculate Variance Covariance Matrix of risk factors

In [54]:
obj_array_util = array_util()
lst_equities_risk_factors = obj_portfolio.getAllRiskFactors()
ser_risk_factors_vol_data = df_risk_factors_data['Volatility']
df_risk_factors_vol_data = pd.DataFrame(ser_risk_factors_vol_data)
mat_portfolio_risk_factors_var_covar = obj_array_util.calc_var_covar_matrix(\
                                obj_portfolio.getAllRiskFactorsNames(), \
                                df_risk_factors_vol_data,\
                                lst_portfolio_stocks_risk_factors_correlation
                                )
                                
obj_portfolio.setPortfolioRiskFactorsVarCovarMatrix(\
                                mat_portfolio_risk_factors_var_covar)

#### Calculate Portfolio Risk

In [55]:
obj_portfolio.calcPortfolioRisk()

[-0.25  0.75  0.5 ]
[[ 0.2  0.9  1.3]
 [ 1.2  0.2  0.7]]


#### Print Inputs

In [56]:
print '*************INPUTS**************'
print '*********************************'
print ''
print df_portfolio_data
print '*********************************'
print df_stocks_beta_data
print '*********************************'
print df_risk_factors_data
print ''

*************INPUTS**************
*********************************

          Weight
Stock Id        
Stock1     -0.25
Stock2      0.75
Stock3      0.50
*********************************
          Beta 1  Beta 2
Stock Id                
Stock1       0.2     1.2
Stock2       0.9     0.2
Stock3       1.3     0.7
*********************************
               Asset Class  Volatility
Risk Factor Id                        
Index1              Equity        0.15
Index2              Equity        0.20



#### Print Outputs

In [57]:
print '************OUTPUTS**************'
print '*********************************'
print ''
print '************Net Betas of stocks in portfolio************'
print obj_portfolio.getAllRiskFactorsNames()
print obj_portfolio.getPortfolioBeta()    
print '********************************************************'
print '***********Covariance Matrix of risk factors************'
print mat_portfolio_risk_factors_var_covar
print '*********Portfolio Variance due to risk factors*********'
print obj_portfolio.getPortfolioVariance()
print '*********Portfolio Volatility due to risk factors*********'
print obj_portfolio.getPortfolioVolatility()

************OUTPUTS**************
*********************************

************Net Betas of stocks in portfolio************
['Index2', 'Index1']
[ 1.275  0.2  ]
********************************************************
***********Covariance Matrix of risk factors************
        Index2  Index1
Index2   0.040 -0.0150
Index1  -0.015  0.0225
*********Portfolio Variance due to risk factors*********
0.058275
*********Portfolio Volatility due to risk factors*********
0.241402154091
